In [51]:
from bs4 import BeautifulSoup
import re
import logging
import datetime
import pandas as pd
import sqlite3
LOG_FILENAME = 'html_extract.log'
logging.basicConfig(filename=LOG_FILENAME,level=logging.DEBUG)



df = pd.read_csv("CaseNumber.csv")

file = open("test.htm",'r')
soup = BeautifulSoup( file, "html.parser")
tds = soup.select('td')

x = ""
for td in tds :
    if "Summons" in td.text:
        x = td

In [35]:
objs = list(x.parent)

In [57]:
# Date
datetime.datetime.strptime(objs[0].text, '%m/%d/%Y').strftime('%Y-%m-%d')

'2017-03-28'

In [37]:
# Href
objs[1].find_all("a", href=True)[0]["href"]

'48960120.pdf'

In [39]:
tds = soup.select('td')

y = ""
for td in tds :
    if  re.search("return on writ", td.text, re.IGNORECASE):
        y = td

In [40]:
y

<td valign="top"><a href="49121667.pdf"><img align="middle" border="0" height="15" src="doc.gif" width="13"/> SHERIFF'S RETURN ON WRIT</a></td>

In [49]:
## Last
tds[-5]

<td valign="top"><a href="48606331.pdf"><img align="middle" border="0" height="15" src="doc.gif" width="13"/> ORDER OF DISMISSAL W/OUT PREJUDICE</a></td>